In [1]:
!ls

bin	docker	     Readme.md	       trained_tokenizers
cfg.py	notebooks    requirements.txt  train_tokenizer.ipynb
config	__pycache__  src


In [2]:
import datasets

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/stephen/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
lang_dataset = datasets.load_from_disk("/home/stephen/Desktop/MEGA_CORPUS/COMBINED_CORPUS/HINDI")

In [4]:
lang_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 65280806
    })
    test: Dataset({
        features: ['text'],
        num_rows: 328045
    })
})

In [5]:
from src.make_datasets import make_sentence_files
make_sentence_files(lang_dataset["train"])

writing to  work/data/hi_sentences/sent_0.txt


100%|██████████| 10000000/10000000 [51:37<00:00, 3227.94it/s] 


writing to  work/data/hi_sentences/sent_1.txt


100%|██████████| 10000000/10000000 [52:04<00:00, 3200.08it/s] 


writing to  work/data/hi_sentences/sent_2.txt


...

100%|██████████| 10000000/10000000 [52:22<00:00, 3181.77it/s] 


writing to  work/data/hi_sentences/sent_5.txt


100%|██████████| 10000000/10000000 [51:23<00:00, 3242.57it/s] 


writing to  work/data/hi_sentences/sent_6.txt


100%|██████████| 5280806/5280806 [27:19<00:00, 3221.18it/s] 


In [2]:
import cfg
from src.make_datasets import sample_and_make_tempfile

tempfile_path = sample_and_make_tempfile(
                                        sentences_dir = cfg.HI_SENTENCES_DIR
                                        , num_files = 7)

sampled files:
['work/data/hi_sentences/sent_2.txt', 'work/data/hi_sentences/sent_1.txt', 'work/data/hi_sentences/sent_0.txt', 'work/data/hi_sentences/sent_4.txt', 'work/data/hi_sentences/sent_3.txt', 'work/data/hi_sentences/sent_6.txt', 'work/data/hi_sentences/sent_5.txt']
number of lines sampled: 18135714


100%|██████████| 18135714/18135714 [08:32<00:00, 35371.14it/s]


Wrote to  work/data/temp.txt


In [ ]:
import sentencepiece as spm
import time

# train for these sizes
vocab_sizes = [8000, 16000, 32000, 48000]

def train_jp(vocab_size):

    start = time.time()
    model_prefix = "cc100_hi" + "_vocab_" + str(vocab_size)
    spm.SentencePieceTrainer.train(input=tempfile_path
                                           , model_prefix=model_prefix
                                           , vocab_size=vocab_size
                                           , character_coverage =1
                                           , num_threads=12
                                           , train_extremely_large_corpus=True
                                          ) 
    print("Trained {} in {} seconds".format(model_prefix, time.time()-start))

# train
for vocab_size in vocab_sizes:
    train_jp(vocab_size)